## Precarga de librerias y funciones

In [28]:
%run "../recurrentes.ipynb"

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.
The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Note: you may need to restart the kernel to use updated packages.


In [2]:
%run "../funciones.ipynb"

In [3]:
# 1 configuración de la base de datos

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [ ]:
# 2 Preseteo optuna especificar bucket y nombre del estudio


nombre_archivo = 'optimization_tree.db'
bucket = 'b1/'


estudio_optuna = base_path + 'buckets/' + bucket + 'optimization_tree.db'


# cargar estudio
# a) competencia_02
# b) competencia_02_lags
# c) competencia_02_lags_y_deltas
# b1 -- competencia_02_lags_deltas_psi_fe
# b1 - 'competencia_02_lags_deltas_psi_fe_sorted'

nombre_estudio = 'competencia_02_lags_deltas_psi_fe'


In [4]:
# 3 - Cargar datos

# Opciones:

# a) dataset_clase_ternaria
# b) dataset_lags_clase_ternaria
# c) dataset_lags_deltas_y_clase_ternaria

df_train = pd.read_parquet(dataset_lags_deltas_y_clase_ternaria_l)
dataset = df_train.copy()



In [6]:
df_train = df_train.drop(columns=dicc_psi['dataset_lags_deltas_y_clase_ternaria'])
df_train = df_train.drop(columns=dicc_psi['lista_light_gbm_feature_importance'])


# Light GBM

## Formateo pre modelo

In [7]:
if 'T_Visa_normal' in df_train.columns:
    df_train['T_Visa_normal'] = df_train['T_Visa_normal'].astype(bool)
if 'T_Master_normal'in df_train.columns:
    df_train['T_Master_normal'] = df_train['T_Master_normal'].astype(bool)


In [8]:
# 4 - parametros para modelo
ganancia_acierto = 273000
costo_estimulo = 7000

mes_train = 202106
mes_test = 202108


In [9]:
data = df_train

In [10]:

data['clase_peso'] = 1.0
data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001
data['clase_binaria'] = np.where(data['clase_ternaria']=='CONTINUA', 0, 1)
df_train = data[data['foto_mes']<=mes_train]
df_test = data[data['foto_mes']==mes_test]
clase_peso = df_train['clase_peso']
X_train = df_train.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_train =df_train['clase_binaria']
X_test = df_test.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_test =df_test['clase_binaria']
w_train = df_train.loc[X_train.index, 'clase_peso']


### Optimización

#### Optimización original

In [ ]:
# 6 - Optimizacion original

def objective(trial):
    num_leaves = trial.suggest_int('num_leaves', 200, 10000)
    learning_rate = trial.suggest_float('learning_rate', 0.005, 0.3)  # mas bajo, más iteraciones necesita
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 1, 8000)
    feature_fraction = trial.suggest_float('feature_fraction', 0.3, 1.0)
    bagging_fraction = trial.suggest_float('bagging_fraction', 0.3, 1.0)

    params = {
        'objective': 'binary',
        'metric': 'custom',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'min_data_in_leaf': min_data_in_leaf,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'seed': semillas[0],
        'verbose': -1
    }

    train_data = lgb.Dataset(X_train,
                             label=Y_train,  # elegir la clase
                             weight=w_train)
                                
    
    cv_results = lgb.cv(
        params,
        train_data,
        num_boost_round=100, # modificar, subit y subir... y descomentar la línea inferior
        # early_stopping_rounds= int(50 + 5 / learning_rate),
        feval=lgb_gan_eval,
        stratified=True,
        nfold=5,
        seed=semillas[0]
    )

    max_gan = max(cv_results['valid gan_eval-mean'])
    best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

    # Guardamos cual es la mejor iteración del modelo
    trial.set_user_attr("best_iter", best_iter)

    return max_gan * 5

storage_name = rf"sqlite:///{estudio_optuna}"

study_name = nombre_estudio



study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

import optuna

class EarlyStoppingByImprovement:
    def __init__(self, patience: int):
        """
        Args:
            patience (int): Número de pruebas consecutivas sin mejora antes de detener la optimización.
        """
        self.patience = patience
        self.best_value = None
        self.no_improvement_trials = 0

    def __call__(self, study, trial):
        current_best_value = study.best_trial.value

        # Si la métrica mejora, reseteamos el contador
        if self.best_value is None or current_best_value > self.best_value:
            self.best_value = current_best_value
            self.no_improvement_trials = 0
        else:
            # Si no hay mejora, incrementamos el contador
            self.no_improvement_trials += 1

        # Detener si no ha habido mejora en 'self.patience' pruebas consecutivas
        if self.no_improvement_trials >= self.patience:
            print(f"Early stopping: No hay mejora en {self.patience} pruebas consecutivas.")
            study.stop()

# Crear el callback con 100 pruebas consecutivas sin mejora
early_stopping_callback = EarlyStoppingByImprovement(patience=100)

# Crear el estudio
study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

# Optimizar el estudio con el callback personalizado
study.optimize(objective, n_trials=30000)

[I 2024-11-08 11:24:00,504] Using an existing study with name 'competencia_02' instead of creating a new one.
[I 2024-11-08 11:24:00,548] Using an existing study with name 'competencia_02' instead of creating a new one.
[I 2024-11-08 11:25:26,175] Trial 1 finished with value: 484022000.0 and parameters: {'num_leaves': 73, 'learning_rate': 0.048381168713106194, 'min_data_in_leaf': 1888, 'feature_fraction': 0.9611290073992866, 'bagging_fraction': 0.5702627562564301}. Best is trial 1 with value: 484022000.0.
[I 2024-11-08 11:26:34,198] Trial 2 finished with value: 454993000.0 and parameters: {'num_leaves': 28, 'learning_rate': 0.03703877249854557, 'min_data_in_leaf': 393, 'feature_fraction': 0.2450786254076267, 'bagging_fraction': 0.23471371488088055}. Best is trial 1 with value: 484022000.0.
[I 2024-11-08 11:28:04,561] Trial 3 finished with value: 489223000.0 and parameters: {'num_leaves': 79, 'learning_rate': 0.05486123634370735, 'min_data_in_leaf': 1380, 'feature_fraction': 0.618491354

KeyboardInterrupt: 

#### Optimización para series temporales con TimeSeriesSplit  

#### Optimización para series temporales con TimeSeriesSplit y undersampling


In [ ]:
best_params = {'num_leaves': 4259,
 'learning_rate': 0.03459780778027798,
 'min_data_in_leaf': 2863,
 'feature_fraction': 0.7029328587219807,
 'bagging_fraction': 0.7914082839316137}

In [ ]:
# 6_ b_Optimización para series temporales con TimeSeriesSplit y undersampling

n_splits = 5
tscv = TimeSeriesSplit(n_splits=n_splits)
folds = list(tscv.split(X_train))

def objective(trial):
    num_leaves = trial.suggest_int('num_leaves', 200, 10000)
    learning_rate = trial.suggest_float('learning_rate', 0.005, 0.3)
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 1, 8000)
    feature_fraction = trial.suggest_float('feature_fraction', 0.3, 1.0)
    bagging_fraction = trial.suggest_float('bagging_fraction', 0.3, 1.0)

    params = {
        'objective': 'binary',
        'metric': 'custom',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'min_data_in_leaf': min_data_in_leaf,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'seed': semillas[0],
        'verbose': -1
    }

    # Asegurarse de que los datos estén ordenados cronológicamente
    X_train_sorted = X_train.sort_index()
    Y_train_sorted = Y_train.loc[X_train_sorted.index]
    w_train_sorted = w_train.loc[X_train_sorted.index]
    
    

    train_data = lgb.Dataset(
        X_train_sorted,
        label=Y_train_sorted,
        weight=w_train_sorted
    )

    cv_results = lgb.cv(
        params,
        train_data,
        num_boost_round=5000,   # Un número muy alto inicial
        feval=lgb_gan_eval,
        folds=folds,
        stratified=False,
        seed=semillas[0],
        early_stopping_rounds=50  # Detenerse si no hay mejora en 50 rondas consecutivas
    )
    max_gan = max(cv_results['valid gan_eval-mean'])
    best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

    # Guardamos la mejor iteración
    trial.set_user_attr("best_iter", best_iter)

    return max_gan * 5

# Configurar el almacenamiento y el estudio de Optuna
storage_name = rf"sqlite:///{estudio_optuna}"
study_name = nombre_estudio

# Crear el estudio
study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

# Definir el callback para early stopping
class EarlyStoppingByImprovement:
    def __init__(self, patience: int):
        self.patience = patience
        self.best_value = None
        self.no_improvement_trials = 0

    def __call__(self, study, trial):
        current_best_value = study.best_trial.value

        if self.best_value is None or current_best_value > self.best_value:
            self.best_value = current_best_value
            self.no_improvement_trials = 0
        else:
            self.no_improvement_trials += 1

        if self.no_improvement_trials >= self.patience:
            print(f"Early stopping: No hay mejora en {self.patience} pruebas consecutivas.")
            study.stop()

# Crear el callback con paciencia de 100
early_stopping_callback = EarlyStoppingByImprovement(patience=100)

# Configurar el nivel de registro de Optuna (opcional)
optuna.logging.set_verbosity(optuna.logging.INFO)

# Optimizar el estudio con el callback personalizado
study.optimize(
    objective,
    n_trials=30000,
    callbacks=[early_stopping_callback]
)


## Voting GD

In [19]:
best_params = {'num_leaves': 4259,
 'learning_rate': 0.03459780778027798,
 'min_data_in_leaf': 2863,
 'feature_fraction': 0.7029328587219807,
 'bagging_fraction': 0.7914082839316137}

In [12]:
# 7 - Voting promedio de modelos para reducir varianza

best_iter = 279

train_data = lgb.Dataset(X_train,
                            label=Y_train,
                            weight=w_train)

df_voting = pd.DataFrame()

for semilla in semillas:
    params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'first_metric_only': True,
    'boost_from_average': True,
    'feature_pre_filter': False,
    'max_bin': 31,
    'num_leaves': 4259,
    'learning_rate': 0.03459780778027798,
    'min_data_in_leaf': 2863,
    'feature_fraction': 0.7029328587219807,
    'bagging_fraction': 0.7914082839316137,
    'seed': semilla,
    'verbose': 0
}
    
    
    model = lgb.train(params,
                    train_data,
                    num_boost_round=best_iter)
    
    df_voting[f'semilla_{semilla}'] = model.predict(X_test)

df_voting['promedio'] = df_voting.mean(axis=1)
    

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [41]:
df_voting['promedio']

0         0.000945
1         0.000605
2         0.000903
3         0.001437
4         0.000349
            ...   
165437    0.004204
165438    0.003416
165439    0.006946
165440    0.001298
165441    0.001646
Name: promedio, Length: 165442, dtype: float64

## Predicción

In [42]:
# 8 - Predicción del modelo 

y_pred_lgm = df_voting['promedio']
y_pred_labels = (y_pred_lgm > 0.025).astype(int)

In [43]:
y_pred_labels

0         0
1         0
2         0
3         0
4         0
         ..
165437    0
165438    0
165439    0
165440    0
165441    0
Name: promedio, Length: 165442, dtype: int32

In [47]:
numero_de_cliente = X_test['numero_de_cliente']

In [52]:

nombres = model.feature_name()
df_resultado = pd.DataFrame({
    'numero_de_cliente': numero_de_cliente,
    'Predicted': y_pred_labels
}, index=X_test.index)

In [57]:
df_resultado['Predicted']=df_resultado['Predicted'].astype(bool)

## Entrega Kaggle

In [58]:
#Ingresar el path sobre el cual se quiere ingresar el archivo a entregar
entrega = 'entrega_kaggle'
formato = '.csv'
numero = rf"\{entrega}_002{formato}"
entrega_final = df_resultado.to_csv(rf"{entregas_l}{numero}", index=False)
entrega_final_path = entregas_l + numero


In [38]:
cantidad_columnas = 298
message = f"{entrega}, cantidad de columnas en el train: {cantidad_columnas}, modelo: LGBM, mejores parametros: {best_params}, mejor iteracion: {best_iter}"

<!-- ### Kaggle -->

In [39]:

competition= 'DMEyF-2024-Segunda'

In [40]:
kaggle.api.competition_submit(competition=competition, file_name= entrega_final_path, message=message, quiet=False)

ApiException: (403)
Reason: Forbidden
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json', 'Date': 'Thu, 21 Nov 2024 19:40:34 GMT', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Origin': '*', 'Set-Cookie': 'ka_sessionid=ab4bec937f913fb3160f01404765ce49; max-age=2626560; path=/, GCLB=CNS3tqL9kZ6GHhAD; path=/; HttpOnly', 'Vary': 'Accept-Encoding', 'X-Kaggle-MillisecondsElapsed': '45', 'X-Kaggle-RequestId': '2bf6bb65e47705065586cd58d95f54f5', 'X-Kaggle-ApiVersion': '1.6.17', 'X-Kaggle-HubVersion': '0.3.4', 'X-Frame-Options': 'SAMEORIGIN', 'Strict-Transport-Security': 'max-age=63072000; includeSubDomains; preload', 'Content-Security-Policy': "object-src 'none'; script-src 'nonce-r54MG+xoG3ywFGrDQ69Opg==' 'report-sample' 'unsafe-inline' 'unsafe-eval' 'strict-dynamic' https: http:; base-uri 'none'; report-uri https://csp.withgoogle.com/csp/kaggle/20201130; frame-src 'self' https://www.kaggleusercontent.com https://www.youtube.com/embed/ https://polyg

### Puntos de corte

In [ ]:

# predicciones = y_pred_lgm

# X_test['Probabilidad'] = predicciones

# tb_entrega = X_test.sort_values(by='Probabilidad', ascending=False)

# cortes = range(9000,14000,100)

# num_subida_kaggle = 65
# for envios in cortes:
    
#     tb_entrega['Predicted'] = 0
#     tb_entrega.iloc[:envios, tb_entrega.columns.get_loc('Predicted')] = 1
#     resultados = tb_entrega[["numero_de_cliente", 'Predicted']].reset_index(drop=True)
    
#     print("Cantidad de clientes {}".format(envios))
    
#     nombre_archivo = 'entrega_0{}.csv'.format(num_subida_kaggle)
#     entrega_final = os.path.join(path, nombre_archivo)
#     resultados.to_csv(entrega_final, index=False)
    
    
#     cantidad_columnas = df_train.shape[1]
#     message = f"{entrega}, cantidad de columnas en el train: {cantidad_columnas}, modelo: LGBM, mejores parametros: {best_params}, mejor iteracion: {best_iter}, archivo: {entrega_final}, punto de corte: {envios}, optimizado con optuna: {study_name}"
    
#     num_subida_kaggle += 1
    
    
    
#     entrega_final = os.path.join(path, nombre_archivo)   
#     competencia = 'dm-ey-f-2024-primera'
#     try:
#         api.competition_submit(file_name=entrega_final,message=message,competition=competencia)
#     except:
#         print(f"Numero máximo de envios, último envio ={num_subida_kaggle}")

In [ ]:
# df_train.shape

In [ ]:
# nombre_modelo = 'lgbm_e_en_abril_p_en_junio_451_features.txt'
# model.save_model(rf"C:\Users\Admin\Documents\1_Notebook\1_Estudio\1_UBA_Maestria_DS\1_Especializacion\1_Segundo_Semestre\DMEyF\modelos_lgbm\{nombre_modelo}")